# 🚀 AI Multi-Agent Codelab

## Build Your AI Startup Team in 30 Minutes!

Trong codelab này, bạn sẽ học cách:
- Tạo AI Agent biết lập kế hoạch
- Cho Agent sử dụng tools (search, image generation)
- Xây dựng Multi-Agent system với Orchestrator
- Deploy landing page cho startup idea của bạn

---

### 🛠️ Tech Stack

| Tool | Mục đích |
|------|----------|
| DeepAgents (LangChain) | Agent orchestration |
| Gemini 2.0 Flash | LLM backbone |
| Gemini Imagen 3 | Image generation |
| Google AI Studio | Landing page canvas |
| Vercel/Netlify | 1-click deployment |

---
## 📦 Cell 0: Installation

In [ ]:
!pip install deepagents langchain-google-genai google-genai -q

---
## 🔑 Cell 1: Setup & API Key

In [ ]:
import os
from google.colab import userdata

# Lấy API key từ Colab Secrets
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

print("✅ API Key configured!")

---
## 🔧 Cell 2: Imports & Client Setup

In [ ]:
from deepagents import create_deep_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from google import genai
from google.genai import types

# Gemini model cho DeepAgents
model = init_chat_model("google_genai:gemini-2.0-flash")

# Gemini client cho Imagen
gemini_client = genai.Client()

# ============================================
# HELPER FUNCTION: Streaming Output
# ============================================
def stream_agent(agent, messages: list[dict], verbose: bool = True):
    """
    Stream agent output với real-time display.
    
    Args:
        agent: DeepAgent instance
        messages: List of message dicts
        verbose: If True, print intermediate steps
    
    Returns:
        Final response content
    """
    final_content = ""
    
    for chunk in agent.stream({"messages": messages}):
        # Handle different chunk types
        if "messages" in chunk:
            for msg in chunk["messages"]:
                if hasattr(msg, "content") and msg.content:
                    print(msg.content, end="", flush=True)
                    final_content = msg.content
                    
        # Show tool calls if verbose
        if verbose and "agent" in chunk:
            agent_data = chunk.get("agent", {})
            if "messages" in agent_data:
                for msg in agent_data["messages"]:
                    if hasattr(msg, "tool_calls") and msg.tool_calls:
                        for tc in msg.tool_calls:
                            print(f"\n🔧 Tool: {tc['name']} → {tc['args']}", flush=True)
    
    print()  # New line at end
    return final_content

print("✅ Setup complete! (with streaming support)")

---
# 📍 MILESTONE 1: First Agent + Planning (10 phút)

**Mục tiêu:** Tạo agent đầu tiên có khả năng lập kế hoạch + generate Mermaid diagram

> 💡 Agent không chỉ trả lời câu hỏi. Agent **NGHĨ** và **LẬP KẾ HOẠCH**.

In [ ]:
# Planning Agent với DeepAgents
# DeepAgents có sẵn write_todos tool để lập kế hoạch!

planning_agent = create_deep_agent(
    model=model,
    system_prompt="""
    Bạn là Planning Agent. Khi nhận task:
    
    1. Dùng write_todos để chia nhỏ task thành các bước
    2. Tạo Mermaid flowchart diagram để visualize
    3. Output cả plan và diagram
    
    Format Mermaid:
    ```mermaid
    graph TD
        A[Start] --> B[Step 1]
        B --> C[Step 2]
        C --> D[Done]
    ```
    
    Luôn output diagram sau khi lập kế hoạch!
    """
)

# 🔥 Streaming output - xem agent "suy nghĩ" real-time!
print("🤖 Planning Agent đang làm việc...\n")
stream_agent(
    planning_agent,
    [{"role": "user", "content": "Lên kế hoạch build landing page cho startup AI"}]
)

### ✅ Checkpoint 1
**Bạn có thấy Mermaid diagram không?** Giơ tay nếu có!

---

# 📍 MILESTONE 2: Agent + Tools (10 phút)

**Mục tiêu:** Cho agent khả năng gọi tools - generate image với Imagen + search web

> 💡 Agent thông minh nhưng bị 'nhốt' trong text. Tools = cho agent **TAY CHÂN** để tương tác với thế giới thực.

In [ ]:
# Tool 1: Generate image với Imagen
@tool
def generate_image(prompt: str) -> str:
    """Generate hero image từ text prompt sử dụng Gemini Imagen API.
    
    Sử dụng tool này khi cần tạo hình ảnh cho landing page, pitch deck,
    hoặc bất kỳ visual asset nào.
    
    Args:
        prompt: Mô tả chi tiết image cần tạo (tiếng Anh cho kết quả tốt nhất)
    
    Returns:
        Confirmation message với path to saved image
    """
    response = gemini_client.models.generate_images(
        model="imagen-3.0-generate-002",
        prompt=prompt,
        config={"number_of_images": 1}
    )
    image = response.generated_images[0]
    image.image.save("hero_image.png")
    return "✅ Đã tạo hero_image.png - Hãy show cho user!"


# Tool 2: Search web với Google Grounded Search (built-in Gemini)
@tool
def search_web(query: str) -> str:
    """Search web để research competitors, market trends, hoặc thông tin.
    
    Sử dụng Google Search có sẵn trong Gemini API (Grounded Search).
    
    Args:
        query: Câu query tìm kiếm
    
    Returns:
        Kết quả search từ Google
    """
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=query,
        config=types.GenerateContentConfig(
            tools=[types.Tool(google_search=types.GoogleSearch())]
        )
    )
    return response.text


print("✅ Tools defined!")

In [ ]:
# Agent với custom tools
visual_agent = create_deep_agent(
    model=model,
    tools=[generate_image, search_web],
    system_prompt="""
    Bạn là Visual Agent. Nhiệm vụ:
    1. Sử dụng search_web để research thông tin cần thiết (Google Search)
    2. Sử dụng generate_image để tạo visual assets (Imagen)
    
    Luôn mô tả chi tiết khi gọi generate_image để có kết quả tốt nhất.
    Nên dùng tiếng Anh cho prompt image.
    """
)

# 🔥 Streaming output - xem agent gọi tools real-time!
print("🤖 Visual Agent đang làm việc...\n")
stream_agent(
    visual_agent,
    [{"role": "user", "content": "Tạo hero image cho startup AI productivity tool"}],
    verbose=True  # Hiển thị tool calls
)

In [ ]:
# Hiển thị image đã tạo
from IPython.display import Image, display
display(Image(filename='hero_image.png'))

### ✅ Checkpoint 2
**Bạn generate được image chưa?** Show lên nào!

---

# 📍 MILESTONE 3: Personalization Agent (10 phút)

**Mục tiêu:** Agent nhận profile user và gợi ý startup ideas phù hợp

> 💡 AI hay nhất là AI hiểu **BẠN**. Paste profile LinkedIn/Facebook → Agent gợi ý ideas match với skills của bạn.

In [ ]:
# ============================================
# BƯỚC 1: User nhập profile
# ============================================
# Template cho user nhập (hoặc paste từ LinkedIn/Facebook)

user_profile = {
    "name": "Nguyễn Văn A",  # 👈 Thay bằng tên bạn
    "age": 22,
    "profession": "Frontend Developer",  # 👈 Thay bằng nghề của bạn
    "interests": ["AI", "Startup", "Gaming"],  # 👈 Thay bằng interests của bạn
    "bio": """
    [PASTE TỪ LINKEDIN ABOUT SECTION]
    
    Ví dụ: "Passionate frontend developer with 2 years of experience
    in React and TypeScript. Love building beautiful UIs and exploring
    AI technologies. Active in local tech communities."
    """
}

print(f"Profile loaded for: {user_profile['name']}")

In [ ]:
# ============================================
# BƯỚC 2: Tạo Personalization Agent
# ============================================

personalization_agent = create_deep_agent(
    model=model,
    system_prompt="""
    Bạn là Personal Branding Expert & Startup Advisor.
    Dựa vào profile user, bạn sẽ:
    
    1. PHÂN TÍCH STRENGTHS:
       - Kỹ năng từ profession
       - Passion từ interests
       - Unique angle từ bio
    
    2. GỢI Ý 3 STARTUP IDEAS (ranked by fit):
       - Phải leverage được skills của họ
       - Phải align với interests
       - Phải có thể build MVP trong 30 phút
    
    3. TẠO PERSONAL TAGLINE:
       - Ngắn gọn, memorable
       - Thể hiện unique value
    
    4. SUGGEST PITCH STYLE:
       - Tech-savvy: Nhiều technical terms, demo-focused
       - Casual: Storytelling, relatable
       - Mix: Balance cả hai
    
    OUTPUT FORMAT:
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    📊 PROFILE ANALYSIS:
    [Phân tích ngắn gọn]
    
    🎯 TOP 3 STARTUP IDEAS:
    1. [Idea] - Match score: X/10
       💡 Lý do: ...
       🛠️ Tech stack gợi ý: ...
    
    2. [Idea] - Match score: X/10
       💡 Lý do: ...
       🛠️ Tech stack gợi ý: ...
    
    3. [Idea] - Match score: X/10
       💡 Lý do: ...
       🛠️ Tech stack gợi ý: ...
    
    ✨ PERSONAL TAGLINE: "..."
    
    🎤 PITCH STYLE: [Style] - [Lý do]
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    """
)

# ============================================
# BƯỚC 3: Chạy agent với streaming
# ============================================
print("🤖 Personalization Agent đang phân tích profile...\n")
stream_agent(
    personalization_agent,
    [{
        "role": "user", 
        "content": f"Đây là profile của tôi, hãy phân tích và gợi ý startup ideas:\n\n{user_profile}"
    }]
)

In [ ]:
# ============================================
# BƯỚC 4: User chọn idea
# ============================================

chosen_idea = input("Bạn chọn idea số mấy? (1/2/3): ")
print(f"\n✅ Bạn đã chọn idea #{chosen_idea}!")
print("\n👉 Tiếp theo: Build Multi-Agent system cho startup của bạn!")

### ✅ Checkpoint 3
**Bạn đã chọn được startup idea chưa?**

---

# 📍 MILESTONE 4: Multi-Agent + Critic (20 phút)

**Mục tiêu:** Build team agents với orchestration + Critic để improve output

> 💡 1 agent giỏi. Nhưng **TEAM agents = unstoppable**. Và team nào cũng cần 1 người critic để quality control.

```
┌─────────────────────────────────────────────────┐
│                 ORCHESTRATOR                     │
│            (Điều phối toàn bộ flow)              │
└─────────────────┬───────────────────────────────┘
                  │
    ┌─────────────┼─────────────┐
    ▼             ▼             ▼
┌───────┐    ┌───────┐    ┌───────┐
│Planner│ →  │ Pitch │ →  │Critic │ ←── Loop until
└───────┘    └───────┘    └───────┘      score ≥ 16/20
```

In [ ]:
# ============================================
# ĐỊNH NGHĨA CÁC SUB-AGENTS
# ============================================

# Sub-agent 1: Pitch Generator
pitch_subagent = {
    "name": "pitch-generator",
    "description": "Tạo pitch content cho startup. Gọi khi cần generate pitch deck content.",
    "prompt": """
    Bạn là Pitch Expert. Tạo pitch theo 4 PILLARS:
    
    1. EXTREME CLARITY - Một câu mô tả, ai cũng hiểu
    2. OBVIOUS VALUE - "Có nó thì sao? Không có thì sao?"
    3. LOW FRICTION - Demo được trong 30 giây
    4. SMART PACKAGING - Visual storytelling
    
    OUTPUT FORMAT:
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    🎯 ONE-LINER: [Max 10 từ]
    
    😫 PROBLEM:
    [Mô tả pain point trong 2 câu]
    
    ✨ SOLUTION:
    [Mô tả giải pháp trong 2 câu]
    
    🔑 KEY FEATURES:
    1. [Feature 1]
    2. [Feature 2]
    3. [Feature 3]
    
    📊 KEY METRIC: [1 con số cụ thể]
    
    🎬 CTA: [Call to action]
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    """,
    "tools": []
}

# Sub-agent 2: Critic
critic_subagent = {
    "name": "critic",
    "description": "Đánh giá và improve pitch quality. Gọi SAU KHI có pitch content.",
    "prompt": """
    Bạn là Pitch Critic khó tính. Đánh giá theo 4 tiêu chí (1-5 điểm mỗi cái):
    
    1. CLARITY (1-5): Người nghe có hiểu trong 10 giây?
    2. VALUE (1-5): "Có nó thì sao? Không có thì sao?" rõ ràng?
    3. UNIQUE (1-5): Khác gì so với giải pháp hiện tại?
    4. VIBE (1-5): GenZ có thấy cool không?
    
    OUTPUT FORMAT:
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    📊 SCORE: X/20
    
    ✅ ĐIỂM MẠNH:
    - [Điểm 1]
    - [Điểm 2]
    
    ⚠️ CẦN CẢI THIỆN:
    - [Điểm 1]
    - [Điểm 2]
    
    💡 GỢI Ý TAGLINE TỐT HƠN: "..."
    
    🎯 VERDICT: [APPROVED ✓ nếu ≥16/20 | NEEDS IMPROVEMENT nếu <16/20]
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    
    Nếu score < 16: Giải thích CỤ THỂ cần improve gì.
    """,
    "tools": []
}

# Sub-agent 3: Landing Page Builder
landing_page_subagent = {
    "name": "landing-page-builder",
    "description": "Tạo HTML landing page từ pitch content. Gọi SAU KHI pitch được approve.",
    "prompt": """
    Tạo landing page HTML hoàn chỉnh với:
    
    1. Hero section với headline từ pitch
    2. Problem/Solution section
    3. 3 Key Features với icons
    4. CTA button nổi bật
    5. Footer với social links
    
    STYLE REQUIREMENTS:
    - Modern gradient: Purple (#7C3AED) → Green (#10B981)
    - Mobile-first responsive
    - Smooth animations
    - Clean typography (Inter font)
    
    Output FULL HTML code có thể deploy ngay.
    """,
    "tools": []
}

# Sub-agent 4: Social Post Writer
social_subagent = {
    "name": "social-writer",
    "description": "Tạo Facebook post để share kết quả. Gọi CUỐI CÙNG sau khi có landing page.",
    "prompt": """
    Tạo Facebook post cho participant share kết quả:
    
    REQUIREMENTS:
    - Tone: Excited nhưng không cringe
    - Emoji: 3-5 cái, đúng chỗ
    - Max 280 ký tự
    - Hashtags bắt buộc: #DevFest2024 #AIAgents #MultiAgent #GenAI #GoogleAI
    
    TEMPLATE:
    🔥 [Hook - 1 câu gây chú ý]
    
    💡 Ý tưởng: [Tên startup]
    🤖 [Số] AI agents làm việc cho mình
    🌐 Live: [URL placeholder]
    
    [Hashtags]
    """,
    "tools": []
}

print("✅ Sub-agents defined!")

In [ ]:
# ============================================
# TẠO ORCHESTRATOR VỚI TẤT CẢ SUB-AGENTS
# ============================================

orchestrator = create_deep_agent(
    model=model,
    subagents=[
        pitch_subagent,
        critic_subagent,
        landing_page_subagent,
        social_subagent
    ],
    system_prompt="""
    Bạn là Startup Builder Orchestrator. Điều phối team agents theo flow:
    
    WORKFLOW:
    1. Nhận startup idea từ user
    2. Gọi pitch-generator để tạo pitch content
    3. Gọi critic để đánh giá pitch
    4. Nếu score < 16/20: Gọi lại pitch-generator với feedback từ critic
    5. Nếu score ≥ 16/20: Gọi landing-page-builder
    6. Cuối cùng: Gọi social-writer để tạo Facebook post
    
    QUAN TRỌNG:
    - Luôn chờ critic approve (≥16/20) trước khi build landing page
    - Max 3 iterations cho pitch improvement
    - Output cuối cùng phải có: Pitch + Score + HTML + Facebook post
    """
)

print("✅ Orchestrator ready!")

In [ ]:
# ============================================
# CHẠY FULL PIPELINE VỚI STREAMING
# ============================================

# 👇 Thay bằng idea bạn đã chọn ở M3
chosen_idea = "AI-powered study buddy for students"

print("🚀 Orchestrator đang điều phối team agents...\n")
print("=" * 50)

stream_agent(
    orchestrator,
    [{
        "role": "user",
        "content": f"""
        Build startup cho idea: {chosen_idea}
        
        Yêu cầu:
        1. Tạo pitch content
        2. Critic phải approve (≥16/20)
        3. Build landing page HTML
        4. Tạo Facebook post
        
        Bắt đầu!
        """
    }],
    verbose=True  # Xem sub-agent nào đang được gọi
)

### ✅ Checkpoint 4
**Pitch của bạn được Critic approve (≥16/20) chưa?**

---

# 🎮 VIBE CODING GAME: AI Startup in 30 Minutes!

## Workflow

```
┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│ STEP 1      │    │ STEP 2      │    │ STEP 3      │
│ Multi-Agent │ →  │ AI Studio   │ →  │ Deploy      │
│ (Notebook)  │    │ (Canvas)    │    │ (Vercel)    │
└─────────────┘    └─────────────┘    └─────────────┘
```

### Timeline

| Time | Task |
|------|------|
| 0-8 min | Personalization + Planning |
| 8-15 min | Pitch + Critic + Image |
| 15-25 min | AI Studio → HTML → Deploy |
| 25-30 min | Social post + Prepare pitch |

## 🚀 Full Starter Pack (Copy & Use!)

In [ ]:
# ============================================
# FULL STARTUP BUILDER - COPY & USE!
# ============================================

from deepagents import create_deep_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from google import genai
from google.genai import types

model = init_chat_model("google_genai:gemini-2.0-flash")
gemini_client = genai.Client()

# ============================================
# STREAMING HELPER
# ============================================
def stream_agent(agent, messages: list[dict], verbose: bool = True):
    """Stream agent output với real-time display."""
    final_content = ""
    
    for chunk in agent.stream({"messages": messages}):
        if "messages" in chunk:
            for msg in chunk["messages"]:
                if hasattr(msg, "content") and msg.content:
                    print(msg.content, end="", flush=True)
                    final_content = msg.content
                    
        if verbose and "agent" in chunk:
            agent_data = chunk.get("agent", {})
            if "messages" in agent_data:
                for msg in agent_data["messages"]:
                    if hasattr(msg, "tool_calls") and msg.tool_calls:
                        for tc in msg.tool_calls:
                            print(f"\n🔧 Tool: {tc['name']} → {tc['args']}", flush=True)
    
    print()
    return final_content

# ============================================
# TOOLS
# ============================================
@tool
def generate_image(prompt: str) -> str:
    """Generate hero image với Gemini Imagen API."""
    response = gemini_client.models.generate_images(
        model="imagen-3.0-generate-002",
        prompt=prompt,
        config={"number_of_images": 1}
    )
    response.generated_images[0].image.save("hero_image.png")
    return "✅ Đã tạo hero_image.png"

@tool
def search_web(query: str) -> str:
    """Search Google với Gemini Grounded Search."""
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=query,
        config=types.GenerateContentConfig(
            tools=[types.Tool(google_search=types.GoogleSearch())]
        )
    )
    return response.text

# ============================================
# SUB-AGENTS
# ============================================
personalizer_subagent = {
    "name": "personalizer",
    "description": "Phân tích profile user và gợi ý startup ideas phù hợp",
    "prompt": "Bạn là Personal Branding Expert. Nhận profile và gợi ý 3 startup ideas phù hợp với skills & interests.",
    "tools": []
}

planner_subagent = {
    "name": "planner",
    "description": "Tạo MVP brief với MoSCoW method",
    "prompt": "Tạo MVP brief: Problem (1 câu), Target User, MUST-HAVE Features (max 3), Success Metric, Mermaid diagram.",
    "tools": []
}

pitch_subagent = {
    "name": "pitch-generator",
    "description": "Tạo pitch content theo 4 Pillars framework",
    "prompt": "Tạo pitch: One-liner, Problem, Solution, 3 Features, Key Metric, CTA.",
    "tools": []
}

critic_subagent = {
    "name": "critic",
    "description": "Đánh giá pitch theo 4 tiêu chí",
    "prompt": "Đánh giá pitch (1-5 điểm): CLARITY, VALUE, UNIQUE, VIBE. Approve nếu ≥16/20.",
    "tools": []
}

landing_page_subagent = {
    "name": "landing-page-builder",
    "description": "Tạo HTML landing page từ pitch",
    "prompt": "Tạo HTML: Hero, Problem/Solution, 3 Features, CTA. Style: gradient purple #7C3AED → green #10B981.",
    "tools": []
}

social_subagent = {
    "name": "social-writer",
    "description": "Tạo Facebook post để share",
    "prompt": "Tạo Facebook post max 280 ký tự. Hashtags: #DevFest2024 #AIAgents #MultiAgent #GenAI #GoogleAI",
    "tools": []
}

# ============================================
# ORCHESTRATOR
# ============================================
startup_builder = create_deep_agent(
    model=model,
    tools=[generate_image, search_web],
    subagents=[
        personalizer_subagent,
        planner_subagent,
        pitch_subagent,
        critic_subagent,
        landing_page_subagent,
        social_subagent
    ],
    system_prompt="""
    Bạn là AI Startup Builder. Workflow:
    1. Nhận profile user → gọi personalizer → lấy startup idea
    2. Gọi planner → tạo MVP brief + Mermaid diagram
    3. Gọi pitch-generator → tạo pitch content
    4. Gọi critic → đánh giá, loop nếu < 16/20
    5. Dùng generate_image tool → tạo hero image
    6. Gọi landing-page-builder → tạo HTML
    7. Gọi social-writer → tạo Facebook post
    
    Output cuối: Pitch + Score + HTML + Facebook post
    """
)

print("✅ Full Startup Builder ready! (with streaming support)")

In [ ]:
# ============================================
# 🚀 RUN YOUR STARTUP BUILDER (STREAMING)
# ============================================

# 👇 ĐIỀN PROFILE CỦA BẠN
user_profile = {
    "name": "Tên của bạn",
    "age": 22,
    "profession": "Nghề nghiệp của bạn",
    "interests": ["Interest 1", "Interest 2", "Interest 3"],
    "bio": "[PASTE LINKEDIN BIO HERE]"
}

print("🚀 AI Startup Builder đang chạy...\n")
print("📡 Streaming mode: Xem output real-time!\n")
print("=" * 50)

stream_agent(
    startup_builder,
    [{
        "role": "user",
        "content": f"Build startup cho tôi. Profile: {user_profile}"
    }],
    verbose=True
)

---

## 🎨 Deploy với AI Studio

1. Mở **[aistudio.google.com](https://aistudio.google.com)** → Canvas mode
2. Paste prompt:
   ```
   Tạo landing page HTML cho startup:
   [PASTE MVP BRIEF + PITCH CONTENT]
   Style: Modern, gradient purple-green
   Include: Hero, Features, CTA, Footer
   ```
3. Click **Preview** → Xem trực tiếp
4. Download HTML
5. Deploy:
   - **Vercel:** [vercel.com/new](https://vercel.com/new) → Drop HTML file
   - **Netlify:** [app.netlify.com/drop](https://app.netlify.com/drop)

---

## 📋 Ý Tưởng Gợi Ý

| Category | Ý Tưởng |
|----------|--------|
| 🎮 Entertainment | Gaming guide, Meme generator, Playlist AI |
| 📚 Education | Study buddy, Homework helper, Language AI |
| 💼 Productivity | Meeting summarizer, Email writer, Task AI |
| 🛒 E-commerce | Price tracker, Review analyzer, Shopping AI |
| 🏥 Health | Fitness coach, Meal planner, Sleep AI |
| 💕 Social | Dating advisor, Event planner, Friend AI |
| 🎨 Creative | Story writer, Art director, Content AI |
| 🤪 Wildcard | Horoscope AI, Excuse generator, Roast bot |

---

## 📱 Facebook Share Template

```
🔥 Vừa build xong AI Startup trong 30 phút tại #DevFest!

💡 Ý tưởng: [TÊN STARTUP]
🤖 Team AI gồm: [SỐ] agents làm việc cho mình
🌐 Landing page LIVE: [LINK]

Từ idea → pitch → website chỉ trong 30 phút.
Đây là sức mạnh của Multi-Agent AI! 🚀

#DevFest2024 #AIAgents #MultiAgent #GenAI #GoogleAI #AIStartup #BuildInPublic
```

---

## 🔗 Resources

| Resource | Link |
|----------|------|
| DeepAgents Docs | https://docs.langchain.com/oss/python/deepagents |
| Gemini API | https://ai.google.dev |
| AI Studio | https://aistudio.google.com |
| Imagen API | https://cloud.google.com/vertex-ai/docs/generative-ai/image/overview |
| Vercel Deploy | https://vercel.com/new |
| Netlify Drop | https://app.netlify.com/drop |

---

## 🎯 Key Takeaways

1. **Agent = AI biết nghĩ và lập kế hoạch**
2. **Tools = Cho agent khả năng tương tác với thế giới thực**
3. **Multi-Agent = Team AI làm việc cùng nhau**
4. **Critic = Quality control cho output**

> 🔥 *"Startup founders validate ideas trong 2 tuần. Với AI Agents, các bạn làm trong 30 phút."*